In [ ]:
# this notebook is to test run the custom stock trading environment

In [1]:
from getstock import get_stock_data_yf_between_with_indicators

In [1]:
import os
import re

# import libraries for training stable baselines agent
from train_stable_agent import create_stable_agents, evaluate_stable_agent, train_stable_agent, output_stable_agent, full_run

from stable_baselines3.common.env_checker import check_env

# import custom functions and classes
from curatedataset import makegymenv, run_env, full_curate_run

from get_agent import Agent, TradingAlgorithm, overbought_confidence


In [ ]:
full_run("train_config_TSLA.json")

In [2]:
full_curate_run('data_config_TSLA.json', 'trained_stable_agents')

Creating environment
[*********************100%%**********************]  1 of 1 completed


/usr/local/lib/python3.10/dist-packages/ta/utils.py:35: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  series = series_output.fillna(method="ffill").fillna(value)
/usr/local/lib/python3.10/dist-packages/ta/utils.py:35: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  series = series_output.fillna(method="ffill").fillna(value)
/usr/local/lib/python3.10/dist-packages/ta/utils.py:35: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  series = series_output.fillna(method="ffill").fillna(value)
/usr/local/lib/python3.10/dist-packages/ta/utils.py:35: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  series = series_output.fillna(method="ffill").fillna(val

init env with max step:  550
Getting stable agents
Running agent:  a2c
Episode:  0 Timestep: 417  done
Episode:  1 Timestep: 286  done
Episode:  2 Timestep: 104  done
Episode:  3 Timestep: 549  done
Episode:  4 Timestep: 535  done
Episode:  5 Timestep: 399  done
Episode:  6 Timestep: 185  done
Episode:  7 Timestep: 456  done
Episode:  8 Timestep: 288  done
Episode:  9 Timestep: 549  done
Episode:  10 Timestep: 274  done
Episode:  11 Timestep: 549  done
Episode:  12 Timestep: 109  done
Episode:  13 Timestep: 236  done
Episode:  14 Timestep: 162  done
Episode:  15 Timestep: 455  done
Episode:  16 Timestep: 549  done
Episode:  17 Timestep: 332  done
Episode:  18 Timestep: 549  done
Episode:  19 Timestep: 389  done
Episode:  20 Timestep: 193  done
Episode:  21 Timestep: 439  done
Episode:  22 Timestep: 200  done
Episode:  23 Timestep: 549  done
Episode:  24 Timestep: 278  done
Episode:  25 Timestep: 549  done
Episode:  26 Timestep: 247  done
Episode:  27 Timestep: 168  done
Episode:  28 Ti

In [1]:
from train_decision_transformer import full_training_run

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trained_model, trained_params = full_training_run('stock_trade_data', device = device)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Processing data as torch tensor.


IndexError: too many indices for tensor of dimension 2

In [ ]:
import json
with open('stock_trade_data/a2c_TSLA_2019-12-01.json') as f:
    agent_config = json.load(f)

In [9]:
# make gym environment
stock_name = 'TSLA'
start_date = '2018-01-01'
num_days = 365*4
interval = '1d'
indicators = ["Volume", "volume_cmf", "trend_macd", "momentum_rsi", "momentum_stoch_rsi", "trend_sma_fast"]
init_balance = 20000
#data = get_stock_data_yf_between_with_indicators(stock_name, '2019-01-01', '2020-02-01', interval, ['all'])
#norm_env, obs_space, act_space, col, data = makegymenv(stock_name, start_date, num_days, interval, normalize=True, init_balance=init_balance)
env,obs_space, act_space, col, data = makegymenv(stock_name, start_date, num_days, interval, indicators=indicators, normalize=False, init_balance=init_balance)


[*********************100%***********************]  1 of 1 completed


/home/victoru/anaconda3/envs/testpython/lib/python3.10/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/home/victoru/anaconda3/envs/testpython/lib/python3.10/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)
/media/victoru/B612CEC512CE8A37/stonkdecisiondocker/stonkdecision/src/curatedataset.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data['momentum_stoch_rsi'].iloc[:10] = 0.5
/home/victoru/anaconda3/envs/testpython/lib/python3.10/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


init env with max step:  1006


In [4]:
data.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'volume_adi',
       'volume_obv', 'volume_cmf', 'volume_fi', 'volume_em', 'volume_sma_em',
       'volume_vpt', 'volume_vwap', 'volume_mfi', 'volume_nvi',
       'volatility_bbm', 'volatility_bbh', 'volatility_bbl', 'volatility_bbw',
       'volatility_bbp', 'volatility_bbhi', 'volatility_bbli',
       'volatility_kcc', 'volatility_kch', 'volatility_kcl', 'volatility_kcw',
       'volatility_kcp', 'volatility_kchi', 'volatility_kcli',
       'volatility_dcl', 'volatility_dch', 'volatility_dcm', 'volatility_dcw',
       'volatility_dcp', 'volatility_atr', 'volatility_ui', 'trend_macd',
       'trend_macd_signal', 'trend_macd_diff', 'trend_sma_fast',
       'trend_sma_slow', 'trend_ema_fast', 'trend_ema_slow',
       'trend_vortex_ind_pos', 'trend_vortex_ind_neg', 'trend_vortex_ind_diff',
       'trend_trix', 'trend_mass_index', 'trend_dpo', 'trend_kst',
       'trend_kst_sig', 'trend_kst_diff', 'trend_ichimoku_conv',
       

In [5]:
col

['Open',
 'High',
 'Low',
 'Close',
 'volume_cmf',
 'trend_macd',
 'momentum_rsi',
 'momentum_stoch_rsi',
 'Balance',
 'Net_worth',
 'Shares_held',
 'Cost_basis',
 'Total_shares_sold',
 'Total_sales_value']